**Data Collection**

**Owner:- Kirpal Singh**
**Contributer:- Arbazkhan Pathan**


**We are collecting the data from the Google Earth Engine using their API and we are collecting the following water parameters :-**

**• Dissolved Oxygen :-** Dissolved oxygen (DO) is a critical parameter in water quality assessment. DO is the amount of oxygen gas that is dissolved in water, and it is essential for aquatic life, particularly for fish and other organisms that require oxygen to respire. When the DO levels in water are low, it can lead to a decline in aquatic life as fish and other aquatic organisms struggle to breathe. Low DO levels in water can also indicate the presence of pollutants and other contaminants that can harm aquatic life and the environment.

**• Salinity :-** Salinity is the measure of the total amount of dissolved salts in water, and it can have significant effects on water quality.High levels of salinity can negatively impact aquatic ecosystems, including plants, animals, and microorganisms that live in the water.

**• Temperature :-** Temperature can have significant effects on the quality of water. Temperature changes can alter the physical and chemical properties of water, affecting the health of aquatic ecosystems and the safety of drinking water.

**• pH :-** pH is a critical parameter that can affect the quality of water. It is a measure of the acidity or basicity of water and can affect the behavior of aquatic organisms and the solubility of chemicals in water.Changes in pH can cause stress or even death in aquatic organisms. Fish and other aquatic organisms have specific pH requirements, and any significant deviation from this range can cause harm to their health and survival.

**• Turbidity :-** Turbidity is a measure of the clarity of water and refers to the presence of suspended particles in water, such as sediment, organic matter, and microorganisms.High turbidity can reduce light penetration in water, which can limit the growth of aquatic plants and affect photosynthesis in aquatic organisms.

**• Dissolved Organic Matter :-** Dissolved organic matter (DOM) refers to organic compounds that are dissolved in water, including compounds from decaying plants and animals, as well as natural organic matter. DOM can react with disinfectants used in water treatment, such as chlorine, to form disinfection byproducts (DBPs), which can have potential health impacts.

**• Suspended Matter :-** Suspended matter in water refers to the presence of solid particles, such as sand, silt, and organic matter, that are suspended in water. Suspended matter can reduce water clarity, making it appear cloudy or murky, which can affect the aesthetic quality of water and reduce its suitability for recreational activities.

**• Nitrate :-** Nitrate is a naturally occurring compound in water that is essential for plant growth, but high levels of nitrate in water can affect water quality and have potential health impacts. Nitrate is a nutrient that can contribute to the growth of algae and aquatic plants in surface waters. High levels of nitrate can lead to eutrophication, a process that results in excessive plant growth, reduced oxygen levels, and potential harm to aquatic organisms.

**• Phosphate :-** Phosphates are essential nutrients that are necessary for the growth and development of aquatic plants and organisms. However, high levels of phosphates in water can affect water quality and have potential health impacts. Phosphates can leach into groundwater from agricultural and industrial activities, septic systems, and other sources. High levels of phosphates in groundwater can contaminate drinking water wells, leading to potential health risks.

**• Chlorophyll :-** Chlorophyll is a pigment that is essential for photosynthesis in aquatic plants and algae. The presence of chlorophyll in water can be an indicator of the level of primary productivity in an aquatic ecosystem.  High levels of chlorophyll in water can indicate high levels of primary productivity, which can lead to eutrophication, a process that results in excessive plant growth, reduced oxygen levels, and potential harm to aquatic organisms.


In [3]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import geemap
import ee

In [4]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AVHEtk7cCl007KACBlWp8IlzhU41sj5H3kO3vsVFlWJGlnHCzrSc63vLLy4

Successfully saved authorization token.


In [5]:
Map = geemap.Map()
geometry = ee.Geometry.Point([76.4968,31.4047])
image = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(geometry) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .first()
start_date, end_date = "2010-01-01", "2020-01-01"

ndwi = image.normalizedDifference(['B3', 'B8'])

ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
ndwiMasked1 = ndwiMasked.toInt()

vectors = ndwiMasked1.reduceToVectors(
    scale=30.0, geometryType='polygon', eightConnected=False, maxPixels=10000000, bestEffort=True)


In [6]:
geometry = ee.Geometry.Point([76.4968,31.4047])
Map.addLayer(geometry)
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(vectors) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median()

visualization = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}
Map.addLayer(sentinel, visualization, 'sent2rgb')

mndwi = sentinel.normalizedDifference(['B3', 'B11']).rename('mndwi')
mndwitr = mndwi.gt(0)
ndsi = sentinel.normalizedDifference(['B11', 'B12']).rename('ndsi')
ndsi2 = sentinel.normalizedDifference(
    ['B11', 'B12']).rename('ndsi2').mask(mndwitr)
Map.addLayer(ndsi2, {'min': 0.1, 'max': 0.4, 'palette': [
             'cyan', 'orange', 'red']}, 'salinity')
ndti = sentinel.normalizedDifference(['B4', 'B3']).rename('ndti')
ndti2 = sentinel.normalizedDifference(
    ['B4', 'B3']).rename('ndti2').mask(mndwitr)
Map.addLayer(ndti2, {'min': -1, 'max': +1,
             'palette': ['blue', 'pink', 'brown']}, 'turbidity')

ndci = sentinel.normalizedDifference(['B5', 'B4']).rename('ndci')
ndci2 = sentinel.normalizedDifference(
    ['B5', 'B4']).rename('ndci2').mask(mndwitr)
Map.addLayer(ndci2, {'min': -1, 'max': +1,
             'palette': ['green', 'pink', 'brown']}, 'chlorophyll')

nitrate = sentinel.expression('(B4 + B3) / (B4 - B3)', {
    'B3': sentinel.select('B3'),
    'B4': sentinel.select('B4')}).rename('nitrate')
Map.addLayer(nitrate, {'min': -1, 'max': 1,'palette': ['blue', 'white', 'green']}, 'nitrate')

phosphate = sentinel.expression('(B4 - B5) / (B4 + B5)', {
    'B4': sentinel.select('B4'),
    'B5': sentinel.select('B5')}).rename('phosphate')
Map.addLayer(phosphate, {'min': -1, 'max': 1,'palette': ['brown', 'white', 'green']}, 'phosphate')


ph = ee.Image(8.339).subtract(ee.Image(0.827).multiply(
    sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph')
ph2 = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select(
    'B1').divide(sentinel.select('B8')))).rename('ph2').mask(mndwitr)
Map.addLayer(ph2, {'min': 0, 'max': 14, 'palette': [
             'red', 'yellow', 'cyan']}, 'ph')

dissolvedoxygen = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select(
    'B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen')
dissolvedoxygen2 = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(
    ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen2').mask(mndwitr)
Map.addLayer(dissolvedoxygen2, {'min': 6.5, 'max': 8, 'palette': [
             'red', 'green', 'blue']}, 'do')



In [7]:
col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(start_date, end_date) \
    .filterBounds(vectors).median()

temp = col.select(
    'ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15).rename('temp')


In [8]:
starting = '2021-01-01'
ending = '2021-08-31'

data = ee.ImageCollection(
    'COPERNICUS/S3/OLCI').filterDate(starting, ending).filterBounds(vectors)

rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
    .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(vectors)


In [9]:
dm_2021_Jan_August_test = rgb.select('Oa08_radiance').divide(
    rgb.select('Oa04_radiance')).rename('dom')
dom2 = rgb.select('Oa08_radiance').divide(
    rgb.select('Oa04_radiance')).mask(mndwitr)
Map.addLayer(dom2, {'min': 0, 'max': 0.8, 'palette': [
             'green', 'red', 'yellow']}, 'Dissolved organic matter')
suspended_matter_2021_Jan_August_test = rgb.select('Oa08_radiance').divide(
    rgb.select('Oa06_radiance')).rename('suspended_matter')
suspended_matter2 = rgb.select('Oa08_radiance').divide(
    rgb.select('Oa06_radiance')).mask(mndwitr)
Map.addLayer(suspended_matter2, {'min': 0, 'max': 0.8, 'palette': [
    'green', 'red', 'yellow']}, 'suspended_matter')


In [10]:
latlon = ee.Image.pixelLonLat().addBands(dm_2021_Jan_August_test)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100,
    tileScale=16)
data_dom_2021_Jan_August_test = np.array((ee.Array(latlon.get("dom")).getInfo()))



In [11]:
latlon = ee.Image.pixelLonLat().addBands(suspended_matter_2021_Jan_August_test)
latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100,
    tileScale=16)

data_sm_2021_Jan_August_test = np.array(
    (ee.Array(latlon.get("suspended_matter")).getInfo()))


In [12]:
latlon = ee.Image.pixelLonLat().addBands(temp)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)

data_lst = np.array((ee.Array(latlon.get("temp")).getInfo()))


In [13]:
latlon = ee.Image.pixelLonLat().addBands(ndti)
latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_ndti = np.array((ee.Array(latlon.get("ndti")).getInfo()))


In [14]:
latlon = ee.Image.pixelLonLat().addBands(ndsi)
latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_ndsi = np.array((ee.Array(latlon.get("ndsi")).getInfo()))


In [15]:
latlon = ee.Image.pixelLonLat().addBands(ndci)
latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_ndci = np.array((ee.Array(latlon.get("ndci")).getInfo()))


In [16]:
latlon = ee.Image.pixelLonLat().addBands(dissolvedoxygen)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100,
    tileScale=16)

data_do = np.array((ee.Array(latlon.get("dissolvedoxygen")).getInfo()))


In [17]:
latlon = ee.Image.pixelLonLat().addBands(ph)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_ph = np.array((ee.Array(latlon.get("ph")).getInfo()))


In [18]:
latlon = ee.Image.pixelLonLat().addBands(nitrate)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_nitrate = np.array((ee.Array(latlon.get("nitrate")).getInfo()))


In [19]:
latlon = ee.Image.pixelLonLat().addBands(phosphate)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_phosphate = np.array((ee.Array(latlon.get("phosphate")).getInfo()))


In [20]:
df = pd.concat([pd.DataFrame(data_do, columns=['Dissolved Oxygen']),
                pd.DataFrame(data_ndsi, columns=['Salinity']),
                pd.DataFrame(data_lst, columns=['Temperature']),
                pd.DataFrame(data_ph, columns=['pH']),
                pd.DataFrame(data_ndti, columns=['Turbidity']),
                pd.DataFrame(data_dom_2021_Jan_August_test,
                             columns=['Dissolved Organic Matter']),
                pd.DataFrame(data_sm_2021_Jan_August_test,
                             columns=['Suspended Matter']),
                pd.DataFrame(data_nitrate,
                             columns=['Nitrate']),    
                pd.DataFrame(data_phosphate,
                             columns=['Phosphate']),         
                pd.DataFrame(data_ndci, columns=['Chlorophyll'])], axis=1, sort=False)



In [21]:
df.head()

,Dissolved Oxygen,Salinity,Temperature,pH,Turbidity,Dissolved Organic Matter,Suspended Matter,Nitrate,Phosphate,Chlorophyll
0,7.645517,0.102493,20.795411,7.000775,-0.321561,0.505255,0.578780,-3.109832,0.002862,-0.002862
1,8.272600,0.114967,20.852093,6.991999,-0.321174,0.520327,0.581242,-3.113579,0.074181,-0.074181
2,5.129700,0.085208,20.826078,6.736983,-0.327032,0.506336,0.563338,-3.057808,0.004929,-0.004929
3,-1.173350,0.114869,20.800110,6.755765,-0.292665,0.507154,0.582198,-3.416870,0.058549,-0.058549
4,3.570700,0.093750,20.877077,7.930263,-0.113924,0.504001,0.564240,-8.777778,-0.026877,0.026877


In [23]:
df.to_csv(r"C:\Users\KIRPAL SINGH\Desktop\Projects\New folder/data.csv")